# Numerical Integration TP (Python)

This notebook covers **basic and intermediate numerical integration** methods,
with plots, convergence study and error tables.

**Methods included**
- Composite **Trapezoid**
- Composite **Midpoint**
- **Simpson** (composite)
- **Monte Carlo**
- SciPy **quad** (adaptive Gauss–Kronrod, used as a reference)
- (optional) **Romberg** via `scipy.integrate.romberg`

**Test functions**
- Polynomial: $f(x)=x^d$
- Gaussian: $f(x)=e^{-x^2}$
- Oscillatory on [0,1]: $f(x)=x^d(1-x)^d\cos x$

We'll compare errors against a high-accuracy reference
(`quad` or an analytic integral when available).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate, special
import pandas as pd

# Plot settings (Matplotlib only; no seaborn, one chart per figure)
%matplotlib inline


In [ ]:
# --- Test functions ---
def f_poly(x, d=2):
    return np.asarray(x)**d

def f_gauss(x):
    return np.exp(-np.asarray(x)**2)

def f_osc(x, d=2):
    x = np.asarray(x)
    return (x**d)*((1-x)**d)*np.cos(x)

# --- Exact/Reference integrals when available ---
def exact_poly(a, b, d=2):
    return (b**(d+1) - a**(d+1)) / (d+1)

def exact_gauss(a, b):
    # integral of exp(-x^2) = sqrt(pi)/2 * (erf(b) - erf(a))
    return np.sqrt(np.pi)/2.0 * (special.erf(b) - special.erf(a))

def quad_ref(f, a, b):
    val, err = integrate.quad(lambda t: float(f(t)), a, b)
    return val


In [ ]:
# --- Numerical integrators (composite rules) ---
def composite_trapezoid(f, a, b, n_panels):
    x = np.linspace(a, b, n_panels+1)
    y = f(x)
    return np.trapz(y, x)

def composite_midpoint(f, a, b, n_panels):
    h = (b - a) / n_panels
    mids = a + (np.arange(n_panels) + 0.5) * h
    return h * np.sum(f(mids))

def composite_simpson(f, a, b, n_panels):
    # Simpson needs an even number of panels (odd number of points)
    if n_panels % 2 == 1:
        n_panels += 1
    x = np.linspace(a, b, n_panels+1)
    y = f(x)
    return integrate.simpson(y, x=x)

def monte_carlo(f, a, b, n_samples, seed=0):
    rng = np.random.default_rng(seed)
    x = a + (b - a) * rng.random(n_samples)
    return (b - a) * np.mean(f(x))


In [ ]:
# --- Quick look at the functions on [0,1] ---
xs = np.linspace(0, 1, 400)
plt.figure()
plt.plot(xs, f_poly(xs, d=2), label="x^2")
plt.plot(xs, f_gauss(xs), label="exp(-x^2)")
plt.plot(xs, f_osc(xs, d=2), label="x^2(1-x)^2 cos x")
plt.title("Test functions on [0,1]")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend()
plt.show()


In [ ]:
# --- Convergence study on [0,1] ---
a, b = 0.0, 1.0
func_name = "gauss"  # "poly" | "gauss" | "osc"
d = 2               # degree for poly/osc

def f(x):
    if func_name == "poly":
        return f_poly(x, d=d)
    elif func_name == "gauss":
        return f_gauss(x)
    else:
        return f_osc(x, d=d)

# reference integral
if func_name == "poly":
    I_ref = exact_poly(a, b, d=d)
elif func_name == "gauss":
    I_ref = exact_gauss(a, b)
else:
    I_ref = quad_ref(f, a, b)  # oscillatory: use quad

Ns = np.array([10, 20, 40, 80, 160, 320, 640])
rows = []
for N in Ns:
    It = composite_trapezoid(f, a, b, N)
    Im = composite_midpoint(f, a, b, N)
    Is = composite_simpson(f, a, b, N)
    Imc = monte_carlo(f, a, b, N, seed=123)
    rows.append({
        "N": N,
        "Trapz": It,
        "Midpoint": Im,
        "Simpson": Is,
        "MonteCarlo": Imc,
        "Error_Trapz": abs(It - I_ref),
        "Error_Midpoint": abs(Im - I_ref),
        "Error_Simpson": abs(Is - I_ref),
        "Error_MonteCarlo": abs(Imc - I_ref),
    })

import pandas as pd
df = pd.DataFrame(rows)
df


In [ ]:
# --- Error plots ---
plt.figure()
plt.loglog(df["N"], df["Error_Trapz"], marker="o", label="Trapezoid")
plt.loglog(df["N"], df["Error_Midpoint"], marker="o", label="Midpoint")
plt.loglog(df["N"], df["Error_Simpson"], marker="o", label="Simpson")
plt.loglog(df["N"], df["Error_MonteCarlo"], marker="o", label="Monte Carlo")
plt.xlabel("N (number of panels / samples)")
plt.ylabel("Absolute error")
plt.title(f"Convergence on [{a},{b}] for {func_name} (d={d})")
plt.legend()
plt.show()


In [ ]:
# --- Compare methods at a fixed computational budget ---
N = 200  # change me
vals = {
    "Trapz": composite_trapezoid(f, a, b, N),
    "Midpoint": composite_midpoint(f, a, b, N),
    "Simpson": composite_simpson(f, a, b, N),
    "MonteCarlo": monte_carlo(f, a, b, N, seed=0),
    "quad (ref)": quad_ref(f, a, b)
}
vals
